In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

In [2]:
df = pd.read_csv('test_data_CANDIDATE.csv', index_col=0)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,nar,hc,sk,trf
index,,,,,,,,,,,,,,,,,
0,49,M,10000,130,269.0,0,1,163,0,0.0,2.0,0,2,2,0,1,6797.761892
1,61,F,10000,138,166.0,0,0,125,1,3.6,NaN,1,2,2,1,3,4307.686943
2,46,F,10000,140,311.0,0,1,120,1,1.8,NaN,2,3,2,0,1,4118.077502
3,69,F,10000,140,254.0,0,0,146,0,2.0,1.0,3,3,2,1,0,7170.849469
4,51,F,10000,100,222.0,0,1,143,1,1.2,1.0,0,2,2,1,0,5579.040145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,54,F,10000,125,273.0,0,0,152,0,0.5,0.0,1,2,2,2,0,6293.123474
284,42,F,10000,120,240.0,1,1,194,0,0.8,0.0,0,3,2,0,1,3303.841931
285,67,M,10000,106,223.0,0,1,142,0,0.3,NaN,2,2,2,1,0,3383.029119


In [3]:
# First of all lets analise the quantity of nulls
df.isna().sum()

age           0
sex           0
cp            0
trestbps      0
chol         16
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope       143
ca            0
thal          0
nar           0
hc            0
sk            0
trf           0
dtype: int64

In [4]:
# Nothing too critical for chol column, but for slope it may be significant, but we still are not sure if we will use
# slope column. Let's replace them with zeroes
df.fillna(0, inplace=True)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,nar,hc,sk,trf
index,,,,,,,,,,,,,,,,,
0,49,M,10000,130,269.0,0,1,163,0,0.0,2.0,0,2,2,0,1,6797.761892
1,61,F,10000,138,166.0,0,0,125,1,3.6,0.0,1,2,2,1,3,4307.686943
2,46,F,10000,140,311.0,0,1,120,1,1.8,0.0,2,3,2,0,1,4118.077502
3,69,F,10000,140,254.0,0,0,146,0,2.0,1.0,3,3,2,1,0,7170.849469
4,51,F,10000,100,222.0,0,1,143,1,1.2,1.0,0,2,2,1,0,5579.040145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,54,F,10000,125,273.0,0,0,152,0,0.5,0.0,1,2,2,2,0,6293.123474
284,42,F,10000,120,240.0,1,1,194,0,0.8,0.0,0,3,2,0,1,3303.841931
285,67,M,10000,106,223.0,0,1,142,0,0.3,0.0,2,2,2,1,0,3383.029119


In [5]:
# Let's check if the most important columns is ok
df['sex'].unique()

array(['M', 'F', 'm', 'f'], dtype=object)

In [6]:
# Let's convert all to caps
df['sex'] = df['sex'].str.upper() 
df['sex'].unique()

array(['M', 'F'], dtype=object)

In [7]:
# Let's check if the dataset it balanced
df.groupby('sex').count()[df.columns[0]]

sex
F    196
M     92
Name: age, dtype: int64

In [8]:
# It is not, let's keep the ratio for later
r = df.groupby('sex').count()[df.columns[0]].values
ratio = r[0] / (sum(r))
ratio

0.6805555555555556

In [9]:
# Let's convert to binary encoding
df.replace('M', 1, inplace=True)
df.replace('F', 0, inplace=True)
df['sex']

index
0      1
1      0
2      0
3      0
4      0
      ..
283    0
284    0
285    1
286    0
287    0
Name: sex, Length: 288, dtype: int64

In [10]:
# Let's check the correlation to see if everything is ok
df.corr()['sex']

age         0.095453
sex         1.000000
cp               NaN
trestbps    0.057594
chol        0.252696
fbs        -0.029891
restecg     0.044233
thalach     0.034308
exang      -0.127497
oldpeak    -0.106737
slope      -0.023036
ca         -0.122047
thal       -0.203524
nar        -0.117275
hc          0.229155
sk         -0.011688
trf        -0.131944
Name: sex, dtype: float64

In [11]:
# There's something unusual, it looks like cp columns values are all the same
df['cp'].unique()

array([10000], dtype=int64)

In [12]:
# Let's eliminate it
del df['cp']

In [13]:
# All correlations are weak, but it doesn't prevend us to being able to use them
c = df.corr()['sex'].abs().sort_values(ascending=False)
c

sex         1.000000
chol        0.252696
hc          0.229155
thal        0.203524
trf         0.131944
exang       0.127497
ca          0.122047
nar         0.117275
oldpeak     0.106737
age         0.095453
trestbps    0.057594
restecg     0.044233
thalach     0.034308
fbs         0.029891
slope       0.023036
sk          0.011688
Name: sex, dtype: float64

In [14]:
# Let's keep the top 3, above 0.2
c = c[c > 0.2]
c

sex     1.000000
chol    0.252696
hc      0.229155
thal    0.203524
Name: sex, dtype: float64

In [15]:
# Let's filter the columns of interest
df_corr = df[c.index]
df_corr

,sex,chol,hc,thal
index,,,,
0,1,269.0,0,2
1,0,166.0,1,2
2,0,311.0,0,3
3,0,254.0,1,3
4,0,222.0,1,2
...,...,...,...,...
283,0,273.0,2,2
284,0,240.0,0,3
285,1,223.0,1,2


In [16]:
# Let's split them into train and test
train_corr = df_corr.loc[:, df_corr.columns != 'sex']
test_corr = df_corr['sex']

In [17]:
# Let's split them into train and test for the model
X_train_corr, X_test_corr, y_train_corr, y_test_corr = train_test_split(train_corr, test_corr, test_size=0.5, random_state=0)

In [18]:
# Let's check the size of the groups
print(len(X_train_corr), len(y_train_corr), len(X_test_corr), len(y_test_corr))

144 144 144 144


In [19]:
# Calculates accuracy, precision, specificity and sensitivity 
def model_rating(clf, X_train, y_train, X_test, y_test, cutoff = 0.5):  

    print('--------------------------Train--------------------------')

    y_p = clf.predict_proba(X_train)[:,1]

    # Compair with cutoff threshold
    y_pred = y_p > cutoff

    # Gets the true negatives, false positives, false negatives and true positives
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()

    # Create dataframe to present data
    results = pd.DataFrame([[tn, fp], [fn, tp]], columns = ['Predicted Female', 'Predicted Male']) 
    results.index = ['Actual Female', 'Actual Male']
    print(results)

    print('---------------------------------------------------------')

    # Calculates results
    accuracy = (tn + tp) / (tn + fp + fn + tp)
    precision = tp / (tp + fp)
    recall_sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Create dataframe to present data
    results = pd.DataFrame([roc_auc_score(y_train, y_p), accuracy, precision, recall_sensitivity, specificity], 
                           columns = ['Values']) 

    results.index = ['ROC AUC Score', 'Accuracy', 'Precision', 'Recall/Sensitivity', 'Specificity']
    print(results)
    print('\n')

    print('--------------------------Test--------------------------')

    y_p = clf.predict_proba(X_test)[:,1]

    # Compair with cutoff threshold
    y_pred = y_p > cutoff

    # Gets the true negatives, false positives, false negatives and true positives
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # Create dataframe to present data
    results = pd.DataFrame([[tn, fp], [fn, tp]], columns = ['Predicted Female', 'Predicted Male']) 
    results.index = ['Actual Female', 'Actual Male']
    print(results)

    print('--------------------------------------------------------')

    # Calculates results
    accuracy = (tn + tp) / (tn + fp + fn + tp)
    precision = tp / (tp + fp)
    recall_sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Create dataframe to present data
    results = pd.DataFrame([roc_auc_score(y_test, y_p), accuracy, precision, recall_sensitivity, specificity], 
                           columns = ['Values']) 

    results.index = ['ROC AUC Score', 'Accuracy', 'Precision', 'Recall/Sensitivity', 'Specificity']
    print(results)

In [20]:
# In order to optimize the parameters of the xg_boost, I've decided to do a grid search
lr = [l/100 for l in list(range(10, 31, 1))] 
print(lr)

parameters = {    
    'max_depth': range (4, 8, 1),
    'n_estimators': range(60, 250, 20),
    'learning_rate': lr,
}

estimator = xgb.XGBClassifier(
    objective= 'binary:logistic',
    scale_pos_weight = ratio,
    seed=0,
    nthread=-1,
)

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = -1,
    verbose=True,
    cv=2,
)

grid_search.fit(X_train_corr, y_train_corr)

[0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3]
Fitting 2 folds for each of 840 candidates, totalling 1680 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 1680 out of 1680 | elapsed:   40.0s finished


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=0.6805555555555556,
                                     seed=0, subsample=None, tree_method=None,
                                     validate_parameters=Non

In [21]:
# Uses the best values obtained on grid search for the model
clf_corr = grid_search.best_estimator_
print(clf_corr)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.19, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=160, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=0.6805555555555556, seed=0, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [22]:
model_rating(clf_corr, X_train_corr, y_train_corr, X_test_corr, y_test_corr, 0.8)

--------------------------Train--------------------------
               Predicted Female  Predicted Male
Actual Female                89               0
Actual Male                  21              34
---------------------------------------------------------
                      Values
ROC AUC Score       0.993871
Accuracy            0.854167
Precision           1.000000
Recall/Sensitivity  0.618182
Specificity         1.000000


--------------------------Test--------------------------
               Predicted Female  Predicted Male
Actual Female               102               5
Actual Male                  22              15
--------------------------------------------------------
                      Values
ROC AUC Score       0.779363
Accuracy            0.812500
Precision           0.750000
Recall/Sensitivity  0.405405
Specificity         0.953271


In [23]:
# Let's try another thing, analising the p value
log = sm.Logit(df['sex'], df.loc[:, df.columns != 'sex']).fit()
log.summary()

Optimization terminated successfully.
         Current function value: 0.502515
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    sex   No. Observations:                  288
Model:                          Logit   Df Residuals:                      273
Method:                           MLE   Df Model:                           14
Date:                Mon, 09 Nov 2020   Pseudo R-squ.:                  0.1978
Time:                        22:06:42   Log-Likelihood:                -144.72
converged:                       True   LL-Null:                       -180.42
Covariance Type:            nonrobust   LLR p-value:                 1.081e-09
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age            0.0269      0.019      1.420      0.156      -0.010       0.064
trestbps       0.0066      0.008      0.782      0.434      -0.010       0.023
chol           0.0104      0.003      3.770      0.000       0.005       0.016
fbs           -0.1163      0.441     -0.264      0.792      -0.981       0.748
restecg        0.2599      0.273      0.952      0.341      -0.275       0.795
thalach        0.0013      0.007      0.197      0.844      -0.012       0.015
exang         -0.3153      0.348     -0.906      0.365      -0.997       0.367
oldpeak       -0.1765      0.150     -1.178      0.239      -0.470       0.117
slope         -0.1395      0.185     -0.756      0.450      -0.501       0.222
ca            -0.3389      0.166     -2.037      0.042      -0.665      -0.013
thal          -0.9149      0.251     -3.644      0.000      -1.407      -0.423
nar           -1.7465      0.696     -2.509      0.012      -3.111      -0.382
hc             0.8928      0.231      3.858      0.000       0.439       1.346
sk            -0.0805      0.148     -0.543      0.587      -0.371       0.210
trf        -7.708e-05   7.46e-05     -1.033      0.302      -0.000    6.92e-05
==============================================================================
"""

In [24]:
# Let's keep the pvalues smaller than 5%
cols = list(df.columns)
cols.remove('sex')
c = ['sex'] + [a for a,b in zip(cols, log.pvalues) if b < 0.05]
c

['sex', 'chol', 'ca', 'thal', 'nar', 'hc']

In [25]:
# Let's filter the columns of interest
df_pvalue = df[c]
df_pvalue

,sex,chol,ca,thal,nar,hc
index,,,,,,
0,1,269.0,0,2,2,0
1,0,166.0,1,2,2,1
2,0,311.0,2,3,2,0
3,0,254.0,3,3,2,1
4,0,222.0,0,2,2,1
...,...,...,...,...,...,...
283,0,273.0,1,2,2,2
284,0,240.0,0,3,2,0
285,1,223.0,2,2,2,1


In [26]:
# Let's split them into train and test
train_pvalue = df_pvalue.loc[:, df_pvalue.columns != 'sex']
test_pvalue = df_pvalue['sex']

In [27]:
# Let's split them into train and test for the model
X_train_pvalue, X_test_pvalue, y_train_pvalue, y_test_pvalue = train_test_split(train_pvalue, test_pvalue, test_size=0.5, random_state=0)

In [28]:
# In order to optimize the parameters of the xg_boost, I've decided to do a grid search
lr = [l/100 for l in list(range(10, 31, 1))] 
print(lr)

parameters = {    
    'max_depth': range (4, 8, 1),
    'n_estimators': range(60, 250, 20),
    'learning_rate': lr,
}

estimator = xgb.XGBClassifier(
    objective= 'binary:logistic',
    scale_pos_weight = ratio,
    seed=0,
    nthread=-1,
)

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = -1,
    verbose=True,
     cv=2,
)

grid_search.fit(X_train_pvalue, y_train_pvalue)

[0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3]
Fitting 2 folds for each of 840 candidates, totalling 1680 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 1528 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 1680 out of 1680 | elapsed:   13.8s finished


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=0.6805555555555556,
                                     seed=0, subsample=None, tree_method=None,
                                     validate_parameters=Non

In [29]:
# Uses the best values obtained on grid search for the model
clf_pvalue = grid_search.best_estimator_
print(clf_pvalue)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.14, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=240, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=0.6805555555555556, seed=0, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [30]:
model_rating(clf_pvalue, X_train_pvalue, y_train_pvalue, X_test_pvalue, y_test_pvalue, 0.6)

--------------------------Train--------------------------
               Predicted Female  Predicted Male
Actual Female                89               0
Actual Male                   9              46
---------------------------------------------------------
                      Values
ROC AUC Score       0.997242
Accuracy            0.937500
Precision           1.000000
Recall/Sensitivity  0.836364
Specificity         1.000000


--------------------------Test--------------------------
               Predicted Female  Predicted Male
Actual Female                95              12
Actual Male                  18              19
--------------------------------------------------------
                      Values
ROC AUC Score       0.766734
Accuracy            0.791667
Precision           0.612903
Recall/Sensitivity  0.513514
Specificity         0.887850


In [31]:
# Let's split them into train and test
train = df.loc[:, df.columns != 'sex']
test = df['sex']

In [32]:
# Since data is umbalanced lets reduce the number of F samples to the number of M samples
rus = RandomUnderSampler(random_state=0)
X_res, y_res = rus.fit_resample(train, test)
print(len(X_res), len(y_res))

184 184


In [33]:
# Let's split them into train and test for the model
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.5, random_state=0)

In [ ]:
# In order to optimize the parameters of the xg_boost, I've decided to do a grid search
lr = [l/200 for l in list(range(10, 61, 1))] 
print(lr)

parameters = {    
    'max_depth': range (4, 8, 1),
    'n_estimators': range(60, 250, 20),
    'learning_rate': lr,
}

estimator = xgb.XGBClassifier(
    objective= 'binary:logistic',
    seed=0,
    nthread=-1,
)

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = -1,
    verbose=True,
    cv=2
)

grid_search.fit(X_train, y_train)

[0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.205, 0.21, 0.215, 0.22, 0.225, 0.23, 0.235, 0.24, 0.245, 0.25, 0.255, 0.26, 0.265, 0.27, 0.275, 0.28, 0.285, 0.29, 0.295, 0.3]
Fitting 2 folds for each of 2040 candidates, totalling 4080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    3.5s


In [ ]:
# Uses the best values obtained on grid search for the model
clf = grid_search.best_estimator_
print(clf)

In [ ]:
model_rating(clf, X_train, y_train, X_test, y_test, 0.8)